In [3]:
pip install numpy scikit-learn plotly


In [4]:
import numpy as np
import plotly.express as px
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler

class FuzzyART:
    def __init__(self, input_dim, rho=0.8, alpha=0.01, beta=1.0):
        self.rho = rho          # Vigilance
        self.alpha = alpha      # Choice parameter
        self.beta = beta        # Learning rate
        self.w = []             # List of weight vectors
        self.input_dim = input_dim

    def _normalize_input(self, x):
        return x / (np.linalg.norm(x) + 1e-6)

    def _choice_function(self, x, w):
        return np.sum(np.minimum(x, w)) / (self.alpha + np.sum(w))

    def _match_function(self, x, w):
        return np.sum(np.minimum(x, w)) / (np.sum(x) + 1e-6)

    def train(self, X):
        labels = []
        for x in X:
            x_norm = self._normalize_input(x)
            chosen = False

            if not self.w:
                self.w.append(x_norm.copy())
                labels.append(0)
                continue

            T = [self._choice_function(x_norm, w) for w in self.w]
            sorted_indexes = np.argsort(T)[::-1]

            for j in sorted_indexes:
                if self._match_function(x_norm, self.w[j]) >= self.rho:
                    self.w[j] = self.beta * np.minimum(x_norm, self.w[j]) + (1 - self.beta) * self.w[j]
                    labels.append(j)
                    chosen = True
                    break

            if not chosen:
                self.w.append(x_norm.copy())
                labels.append(len(self.w) - 1)

        return np.array(labels)

def generate_data():
    X, _ = make_blobs(n_samples=200, centers=4, cluster_std=0.5, random_state=42)
    X = MinMaxScaler().fit_transform(X)
    return X

def plot_clusters(X, labels, title):
    fig = px.scatter(x=X[:, 0], y=X[:, 1], color=labels.astype(str),
                     labels={'x': 'Feature 1', 'y': 'Feature 2'},
                     title=title)
    fig.update_traces(marker=dict(size=6, opacity=0.8, line=dict(width=0.5, color='DarkSlateGrey')))
    fig.show()

def run_vigilance_experiment():
    X = generate_data()
    vigilances = [0.2, 0.5, 0.75, 0.95]

    for rho in vigilances:
        art = FuzzyART(input_dim=X.shape[1], rho=rho)
        labels = art.train(X)
        plot_clusters(X, labels, f'Fuzzy ART Clustering (Vigilance ρ = {rho})')

if __name__ == "__main__":
    run_vigilance_experiment()
